<a href="https://colab.research.google.com/github/davidnoone/GEOPHYS_NOTEBOOKS/blob/main/ENVPHYS200_waether_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENVPHYS 200: Weather mapping lab


Weather maps are used to provide a synopsis of the atmospheric circulation: often called the “synoptic chart”. You will examine the atmospheric conditions in the region of New Zealand for a the time period around 11 February 2023.

Obtain datasets for mean sea level pressure (MSLP), 1000 hPa geopotential height (Z500), 500 hPa Geopotential height (Z1000), total column water content (“precipitable water”, PWTR).

_Your figures should be aesthetically pleasing! The figures should also have titles which descript what quantity is plot, the date, and the units. If you use color filled contours, be sure to include a color bar that describes the numerical value of the scale. For wind vectors, you may wish to plot a subset of all vectors, since there may be too many to have a clear diagram)._


###Learning goals
* (Hydrostatic) Similarity of height at constant pressure, and pressure at constant height
* (Hydrostatic/hypsometric) Relationship between geopotential and temperature
* (Geostrophic) Winds and pressure gradients


###Python goals
* Explore use of gridded 2d data in python (using numpy)
* Plotting maps of 2d data (contour and contourf)
* Plotted maps of 2d vector fields (quiver)
* Intro to map projections, geographic coordinates.




In [ ]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt

try:
    import netCDF4 as ncdf
except:
    print('NetCDF needs to be installed: this will take a few moments')
    !pip install netCDF4
    import netCDF4 as ncdf

try:
  import cartopy.crs as ccrs
except:
  print('cartopy needs to be installed: this will take a few moments')
  !pip install cartopy
  !pip uninstall shapely -y
  !pip install shapely --no-binary shapely
  import cartopy.crs as ccrs


# Download a data file from github, if it doesn't exist
!test ! -f ERA_prl_20230211.nc && wget 'https://github.com/davidnoone/GEOPHYS_NOTEBOOKS/raw/main/ERA_prl_20230211.nc'
!test ! -f ERA_sfc_20230211.nc && wget 'https://github.com/davidnoone/GEOPHYS_NOTEBOOKS/raw/main/ERA_sfc_20230211.nc'



In [ ]:
# Some physical constants which we will need later
gravity = 9.81            # acceleration due to gravity [m/s]
rearth  = 6371000.        # radius of the earth [m]
omega   = 7.2921159e-5    # Rotation rate of earth (2 pi radians/day)


def calculate_gradient(lons,lats,fld):
    """ Uses a finite difference method to compute grradients in x and y
        of some 2d field: fld(y,x), returns dfdx and dfdy
        Assumes "x" is longitudes, and "y" is latitudes
        Assume grid has constant spacing.
    """
    # calculate distance, in meters, between grid points
    dx = rearth*np.radians((lons[2] - lons[1]))
    dy = rearth*np.radians((lats[2] - lats[1]))
    print('DX=',dx,'  DY=',dy)
    coslat = np.cos(np.radians(lats))

    # gradient in "x": use "roll" to account for periodic edges
    dfdx = (np.roll(fld, -1, axis=1) - np.roll(fld,+1,axis=1))     # "df" as a difference
    for j in range(len(coslat)):
        dfdx[j,:] = dfdx[j,:]/(2*dx*coslat[j])                # divide by "dx"


    # gradent in "y"
    dfdy = np.gradient(fld,axis=0)/(2*dy)

    return dfdx, dfdy


##Data
We will use data from the ECMWF 5th Reanalysis ("ERA5"). This data set is a comprehensive global 3d depiction of the atmosphere that spans from 1940 until today! It has developed in a way similar to opperational weather forecasts, by blending observations (from baloons, satellies, surface sites and other) with a state-of-the-art weather model. As a result a large number of variables are available for analysis.

We will focus on...
* Mean sea level pressure (MSLP)
* Geopotential height at 1000 and 500 hPa (Z100 and Z500)
* Precipitable water (PWTR)

Each of these variables exists on a latitude x longitude grid, which has a spacing of 0.25 degrees.

The original data comes in a data format called NetCDF, which is a very common data format for large atmospheric datasets. Python has a library that can read these data.

In [ ]:
# Read in the data

with ncdf.Dataset('ERA_prl_20230211.nc','r') as ncfile:    # open the file and read the data...
  lons  = ncfile.variables['longitude'][:]    # read the longitudes
  lats  = ncfile.variables['latitude'][:]     # read the latitudes
  levs  = ncfile.variables['level'][:]        # read the pressre level
  z500  = ncfile.variables['z'][0,0,:,:]      # read the 500 hPa height
  z1000 = ncfile.variables['z'][0,1,:,:]      # read the 1000 hPa height

with ncdf.Dataset('ERA_sfc_20230211.nc','r') as ncfile:    # open the file and read the data...
  mslp  = ncfile.variables['msl'][0,:,:]      # read the mean sea level pressure
  pwtr  = ncfile.variables['tcw'][0,:,:]      # read the mean sea level pressure

mslp = mslp/100         # convert units from Pa to hPa
z500 = z500/gravity     # convert units from geopotential to metres
z1000 = z1000/gravity   # convert units from geopotential to metres

nlon = len(lons)
nlat = len(lats)
nhalf = int(nlon/2)


Do a little check of the data, to make sure it is what we think it is!



In [ ]:
print("Shape of lats:",np.shape(lats))
print("Shape of lons:",np.shape(lons))
print("Shape of mslp:",np.shape(mslp))

print('Latitudes:',lats)
print('Longitudes:',lons)

---
### Task (a) Make a contour plot of each of the four quantities: MSLP, Z1000, Z500 and PWTR
_The contour plots should have labels and a title that describes what each plot is._

You will use the "contour" and "contourf" functions.
Your figures can have a title.

You may also wish to explore different contour levels, adding a colorbar, adding line labels, and possibly changing the colors!

See documentation and examples at:
https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.contour.html

Particularly useful key words are "levels", "cmap", "colors"



In [ ]:
# Example: plotting mean sea level

extent = [140, 200, -65, -10]
proj_grid = ccrs.PlateCarree(180)      # Map projection with 180 is the central longitude
proj_data = ccrs.PlateCarree(0)      # Data has "longitude", relative to 0 E.


fig = plt.figure(figsize=[12,8])                  # Create a figure

ax = plt.axes(projection=proj_grid)
ax.set_extent(extent,proj_data)
ax.set_title("Mean Sea Level Pressure (hPa) 2023/02/11 00:00:00")

# Shaded contours for temperature
levels = 960 + 2*np.arange(41)       # set values for contour fill
cf = plt.contourf(lons, lats, mslp, levels, cmap='jet',transform=proj_data)

## Add a color bar
plt.colorbar(cf)

# Overlay contour lines
levels = 900 + 4*np.arange(51)       # set values for contour lines
cl = plt.contour(lons, lats, mslp, levels, colors='black',transform=proj_data)

# Add contour line liables
ax.clabel(cl, cl.levels, inline=True, fontsize=10)


# Add costlines
ax.coastlines(color='blue')

# Add grid lines
ax.gridlines(crs=proj_grid, linewidth=1, color='cyan',
    draw_labels=True, alpha=0.5, linestyle='--')



In [ ]:
# Your plots for Z1000, Z500 and PWTR here
# (It is recommended you use the example above, and adapt it.)


# Z1000




# Z500



# PWTR



---
###Task (b)	Describe the visual similarity between MSLP and Z1000.
_Making reference to the hydrostatic equation, give an explanation for your answer._



[Click to type answer here]

---
###Task (c) Make a plot of the difference in the height between 500 hPa – 1000 hPa

In [ ]:
# Your plot of thickness: Units


---
###Task (d): Explain why the 1000-500 hPa thickness is a good indicator of average tropospheric temperature.
_Making reference to the hydrostatic equation in your answer. You may wish to write down the equation_

[Click to type your answer]


---
###Task (e): Calculate and plot the average lower troposphere temperature in units of Kelvin.

Recall the hypsometric equation:

$$
\frac{dp}{dz} = - \rho g
$$

and the ideal gas law ($ p = \rho R T $), can be combined, then integrated to give the hypsometric equation. This relates differences in pressure to differences in height:

$$
\Delta z = z_2 - z_1 = \frac{R T}{g} ln\left( \frac{p_1}{p_2} \right)
$$

Thickness, which depends on the average temperature, T.



In [ ]:
# Your plot of temperature in units of Kelvin



---
### Task (f) Calculate the geostrophic wind at 500 hPa in units of m/s, and plot it.

First you will need to calculate the "u" and "v" wind components, using the geostrophic balance equations.

Recall for geostophic balance, the pressure gradient force is balanced (equal and opposite) to the Coriolis force. As such we can write:

$$
u_{geo} = -\frac{g}{f} \frac{\partial z}{\partial y}
$$

$$
v_{geo} =  \frac{g}{f} \frac{\partial z}{\partial x}
$$


Where f is the Coriolis parameter, and g is the acceleration due to gravity.
The "gradient" terms can be computed on the grid using the idea of finite differences: that is the "slope" between one grid point and the next. The function defined above does this: calculate_gradient.


Once you have these, you can use the "quiver" function to construct the a plot of the little vectors.

You can calculate the wind speed in the usual way.

$$
|V| = \sqrt{(u^2 + v^2)}
$$

Aim to produce a plot similar to the one from part one of the assignment: showing wind speed in color shading, and then direction as little arrow.

You might also find it helpful to overlay a contour lines for the Z500.



In [ ]:
# Calculate the coriolis parameter: f = 2 * omega * sin(latitude)
#fcor = 2.0*omega*np.sin(np.radians(lats))
fcor = -1.e-4       # typical value in southern hemisphere


# Calculate the pressure gradient in "x" and "y".
dzdx, dzdy = calculate_gradient(lons,lats,z500)


# Calculate the geostrophic wind components
ugeo = np.zeros((nlat,nlon))         # YOUR CODE HERE
vgeo = np.zeros((nlat,nlon))         # YOUR CODE HERE



# Calculate the geostriophic wind speed
vspeed = np.sqrt(ugeo*ugeo + vgeo*vgeo)


# PLOT a figure!
fig = plt.figure(figsize=[12,8])                  # Create a figure

ax = plt.axes(projection=proj_grid)
ax.set_extent(extent,proj_data)
ax.set_title("Wind vectors and speed at 500 hPa 2023/02/11 00:00:00")

# Contour fill for speed
cf = plt.contourf(lons, lats, vspeed, 21, cmap='YlOrRd',transform=proj_data)
plt.colorbar(cf)

# "quiver" for vectors:
scale = 10
ax.quiver(lons, lats, ugeo, vgeo,\
              scale_units="xy",scale=scale,transform=proj_data,color='blue')


# Contour lines for Z500
#plt.contour(lons,lats,z500)





---
### Task (g): Examining your diagrams, describe the expected influence of advection on the temperature and water content near Auckland.

[Double click to enter your answer]

---
### Task (h): Describe the weather conditions. What do you expect the weather to be like in the Auckland region in the day or two which follows.

Make reference to the figures, and explain your reasoning.

[Double click to type your answer]

---
# Outcome

Having reached the end, you have now learned to be able to read, visualize, and perform calulations with 2d geophysical datasets. Datasets like these are used in opperational and research applications in many fields of earth science, including weather and climate modeling.

Here, you've been able to combined oberservations, with derived calulations for temperature and wind, that together allow you to offer an analysis of current weather conditions, and how the atmospheric circulation on this day is likely to change.


In [ ]:
#